In [31]:
__author__ = "Angus Liao 2023/2/24"

In [32]:
from gurobipy import *
from gurobipy import quicksum
from gurobipy import Model
from gurobipy import GRB
import pandas as pd
import numpy as np

# 讀入資料

In [33]:
#用pandas的read_csv方法讀入資料，參數是字串型態的檔名
Electric = pd.read_csv("data/整點資料CSV.csv")
Electric

,Date,Time,Price,Theta,Load(kW)
0,2020/1/1,01:00,1.50,0.0,1542
1,2020/1/1,02:00,1.50,0.0,1504
2,2020/1/1,03:00,1.50,0.0,1487
3,2020/1/1,04:00,1.50,0.0,1510
4,2020/1/1,05:00,1.50,0.0,1506
...,...,...,...,...,...
8779,2020/12/31,20:00,3.44,0.0,3174
8780,2020/12/31,21:00,3.44,0.0,2812
8781,2020/12/31,22:00,3.44,0.0,2350
8782,2020/12/31,23:00,1.50,0.0,1859


# 宣告儲存變數用的陣列、宣告變數

In [34]:
#用三維資料進行儲存 [月份][日期][小時]

Price = np.zeros([12, 31, 24])
Theta = np.zeros([12, 31, 24])
Load = np.zeros([12, 31, 24])

In [35]:
#各月份天數
N_m = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# 將資料存成三維陣列[月份][日期][小時]

In [36]:
#陣列從0開始，1月 => month=0；1日 => date=0 : 1時 => hour=0

index_count = 0  #呼叫資料框資料進行儲存用的索引值

for month in range(0, 12):  #12個月
    for date in range(0, N_m[month]):  #不同月份有不同天數，date範圍不同，用N_m陣列值代表範圍
        hour = 0  #新的一天開始存資料，小時索引值要更新
        for index in range(index_count, index_count+24):  #一次存24筆資料(24小時)
            #print(str(month+1) + "/" + str(date+1) + "/" + str(hour+1) + "/" + str(index))  <==測試索引值用，不用理會
            
            #Electric.iloc[index, :] => 用pandas的iloc方法呼叫對應的索引值資料
            #Electric["Price"] => 用pandas語法呼叫對應column name的資料
            #tolist() => 把資料數據轉換為list型態，方便後續轉換為多維陣列
            #np.array => Numpy多維陣列宣告的語法
            Price[month][date][hour] = np.array(Electric.iloc[index, :]["Price"].tolist())
            Theta[month][date][hour] = np.array(Electric.iloc[index, :]["Theta"].tolist())
            Load[month][date][hour] = np.array(Electric.iloc[index, :]["Load(kW)"].tolist())
            hour = hour + 1  #每存一筆資料，小時索引值要加1，代表新的一個小時的資料
        index_count = index_count + 24  #一天存24小時，索引值計數器要加24，代表下次要儲存下一組24筆資料

In [37]:
#計算用戶年度總需求
Total_load = 0

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            Total_load += Load[month][date][hour]

# 宣告各類成本

In [38]:
C_holding = 0.5  #儲能內部持有成本

C_power_generation = 5  #光伏發電成本

C_charge = 10.6  #儲能充電成本

beta = 0.90  #放電效率

# 用Gurobi跑最佳化

In [39]:
#定義模型
opt_mod = Model(name = "Electric_optimize")

# 加入決策變數

In [40]:
#name => 決策變數名稱
#vtype => 決策變數型態
#lb => 變數值下限
E_pv = opt_mod.addVar(name = "E_pv", vtype = GRB.INTEGER, lb = 0)  #光伏容量
E_bat = opt_mod.addVar(name = "E_bat", vtype= GRB.INTEGER, lb = 0)  #儲能容量
Total_pv = opt_mod.addVar(name = "Total_pv", vtype= GRB.INTEGER, lb = 0)  #太陽能總發電量

opt_mod.update()  #每對模型做一次更改都要更新模型一次

In [41]:
Charge, Discharge, Purchase, Inventory, PVoutput = [], [], [], [], []  #用陣列儲存新增決策變數
#充電量、放電量、購電量、期末存貨量、光伏輸出量

for month in range(0, 12):
    for date in range(0, 31):  #後續要將陣列reshape，全數資料數量要一致，不做日期分別
        for hour in range(0, 24):
            #用list型態資料的append方法新增資料
            #name => 決策變數名稱，為了方便結果視覺化，將索引值+1
            Charge.append(opt_mod.addVar(name = "Charge[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", vtype = GRB.INTEGER, lb = 0))
            Discharge.append(opt_mod.addVar(name = "Discharge[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", vtype = GRB.INTEGER, lb = 0))
            Purchase.append(opt_mod.addVar(name = "Purchase[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", vtype = GRB.INTEGER, lb = 0))
            Inventory.append(opt_mod.addVar(name = "Inventory[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", vtype = GRB.INTEGER, lb = 0))
            PVoutput.append(opt_mod.addVar(name = "PVoutput[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", vtype = GRB.INTEGER, lb = 0))


#轉換成numpy陣列
Charge = np.array(Charge)
Discharge = np.array(Discharge)
Purchase = np.array(Purchase)
Inventory = np.array(Inventory)
PVoutput = np.array(PVoutput)

#將陣列reshape成三維陣列，方便後續利用索引值[月份][日期][小時]寫程式
Charge = Charge.reshape(12,31,24)
Discharge = Discharge.reshape(12,31,24)
Purchase = Purchase.reshape(12,31,24)
Inventory = Inventory.reshape(12,31,24)
PVoutput = PVoutput.reshape(12,31,24)

opt_mod.update()

In [42]:
#把不存在的日期的決策變數刪除

for v in opt_mod.getVars():  #取得決策變數名稱，用於判斷日期是否合理
    #需要一個日期一個日期的變數刪
    if "[2][30]" in v.VarName:
        opt_mod.remove(v)
    if "[2][31]" in v.VarName:
        opt_mod.remove(v)
    if "[4][31]" in v.VarName:
        opt_mod.remove(v)
    if "[6][31]" in v.VarName:
        opt_mod.remove(v)
    if "[9][31]" in v.VarName:
        opt_mod.remove(v)
    if "[11][31]" in v.VarName:
        opt_mod.remove(v)
        
opt_mod.update()

# 定義目標函式

In [43]:
obj_function = sum((Charge[month][date][hour]*C_charge) + (PVoutput[month][date][hour]*C_power_generation) + (Inventory[month][date][hour]*C_holding) + (Purchase[month][date][hour]*Price[month][date][hour]) for month in range(0, 12) for date in range(0, N_m[month]) for hour in range(0, 24))
#儲能充電量*充電成本 + 光伏發電量*發電成本 + 儲能內部期末存貨量*持有成本 + 購電量*購電價

In [44]:
#以最小化求解
opt_mod.setObjective(obj_function, GRB.MINIMIZE)

In [45]:
#拿來複製用，不用理會
"""
for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
"""

'\nfor month in range(0, 12):\n    for date in range(0, N_m[month]):  \n        for hour in range(0, 24):\n'

# 加入限制式
## addConstr() => 加入一般限制式，裡面只能==、>=、<=

In [46]:
#各時段光伏輸出量限制:
#光伏容量*各時段光伏發電因數

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            opt_mod.addConstr(PVoutput[month][date][hour] == E_pv*Theta[month][date][hour])
    
opt_mod.update()

In [47]:
#光伏年度總輸出須滿足20%用戶年度負載量

opt_mod.addConstr(Total_pv == quicksum(PVoutput[month][date][hour] for month in range(0, 12) for date in range(0, N_m[month]) for hour in range(0, 24)))
opt_mod.addConstr(Total_pv >= 0.2*Total_load)

opt_mod.update()

In [48]:
#充電量限制式:
#不得超過剩餘容量空間 => 電池容量-前期期末存貨
#若光伏未發電，只放不充 => 充電量=0

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            if month == 0:  #一月
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= Discharge[month][date][hour])  #預設初始存貨是滿的
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat -  Inventory[month][date][hour-1] + Discharge[month][date][hour])  #跨小時，小時索引值要-1
                else:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date-1][23] + Discharge[month][date][hour])  #跨日，日期索引值-1，小時索引值23
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
            elif month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10:  #二、四、六、八、九、十一月第一天的第一時前期是上個月的31日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month-1][30][23] + Discharge[month][date][hour])  #跨月，月索引值-1，日期索引值30(前一個月的31日)，小時索引值23
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date-1][23] + Discharge[month][date][hour])
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
            elif month == 2:  #三月第一天的第一時前期是上個月的29日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month-1][28][23] + Discharge[month][date][hour])  #跨月，月索引值-1，日期索引值28(前一個月的29日)，小時索引值23
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date-1][23] + Discharge[month][date][hour])
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
            else:  #五、七、十、十二月的第一天第一時前期是上個月30日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month-1][29][23] + Discharge[month][date][hour])  #跨月，月索引值-1，日期索引值29(前一個月的30日)，小時索引值23
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date-1][23] + Discharge[month][date][hour])
                    else:
                        opt_mod.addConstr(Charge[month][date][hour] <= E_bat - Inventory[month][date][hour-1] + Discharge[month][date][hour])
                
opt_mod.update()

In [49]:
#放電量限制式:
#不得超過前期期末存貨量
#放電量<=前期期末存貨
#需考慮放電效率、引值變化

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            if month == 0:  #一月
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*E_bat)  #預設初始存貨是滿的
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])  #跨小時，小時索引值要-1
                else:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date-1][23])  #跨日，日期索引值-1，小時索引值23
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
            elif month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10:  #二、四、六、八、九、十一月第一天的第一時前期是上個月的31日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month-1][30][23])  #跨月，月索引值-1，日期索引值30(前一個月的31日)，小時索引值23
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date-1][23])
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
            elif month == 2:  #三月第一天的第一時前期是上個月的29日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month-1][28][23])  #跨月，月索引值-1，日期索引值28(前一個月的29日)，小時索引值23
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date-1][23])
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
            else:  #五、七、十、十二月的第一天第一時前期是上個月30日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month-1][29][23])  #跨月，月索引值-1，日期索引值29(前一個月的30日)，小時索引值23
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
                else:
                    if hour == 0:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date-1][23])
                    else:
                        opt_mod.addConstr(Discharge[month][date][hour] <= beta*Inventory[month][date][hour-1])
                        
opt_mod.update()

In [50]:
#期末存貨限制式:
#若光伏有發電 => 充放皆可: 當期期末存貨=前期期末存貨+當期充電量-當期放電量
#若光伏未發電 => 只放不充: 當期期末存貨=前期期末存貨-當期放電量
#需考慮放電效率、索引值變化

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            if month == 0:  #一月
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == E_bat + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))  #預設初始存貨是滿的
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                else:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date-1][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
            elif month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10:  #二、四、六、八、九、十一月第一天的第一時前期是上個月的31日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month-1][30][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                else:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date-1][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
            elif month == 2:  #三月第一天的第一時前期是上個月的29日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month-1][28][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                else:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date-1][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
            else:  #五、七、十、十二月的第一天第一時前期是上個月30日24時
                if date == 0:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month-1][29][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                else:
                    if hour == 0:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date-1][23] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))
                    else:
                        opt_mod.addConstr(Inventory[month][date][hour] == Inventory[month][date][hour-1] + Charge[month][date][hour] - (Discharge[month][date][hour]/beta))

opt_mod.update()

In [51]:
#系統滿足負載量限制式:
#若光伏有發電 => 充放皆可: 光伏+儲能輸出要滿足20%用戶負載
#若光伏未發電 => 只放不充: 儲能輸出要滿足20%用戶負載

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            if Theta[month][date][hour] == 0:
                opt_mod.addConstr(Discharge[month][date][hour] >= Load[month][date][hour]*0.2)
            else:
                opt_mod.addConstr(PVoutput[month][date][hour] + Discharge[month][date][hour] >= Load[month][date][hour]*0.2)
                
opt_mod.update()

In [52]:
#全時段供給須滿足需求限制式:
#購電量+放電量+光伏輸出>=負載+充電量

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            opt_mod.addConstr(Purchase[month][date][hour] + Discharge[month][date][hour] + PVoutput[month][date][hour] - Load[month][date][hour] - Charge[month][date][hour] >= 0)
            
opt_mod.update()

In [53]:
#儲能容量限制式:
#容量須滿足最大期末存貨量

#Gurobi最大值限制式語法 => (a == [這些資料當中的最大值])
opt_mod.addGenConstrMax(E_bat, [Inventory[month][date][hour] for month in range(0, 12) for date in range(0, N_m[month]) for hour in range(0, 24)])

opt_mod.update()

# 運行最佳化

In [54]:
opt_mod.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 52706 rows, 43923 columns and 158220 nonzeros
Model fingerprint: 0xdadf7610
Model has 1 general constraint
Variable types: 0 continuous, 43923 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 7e+06]
Presolve added 0 rows and 8783 columns
Presolve removed 6919 rows and 0 columns
Presolve time: 0.40s
Presolved: 45787 rows, 52706 columns, 157504 nonzeros
Presolved model has 8784 SOS constraint(s)
Variable types: 0 continuous, 52706 integer (8784 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.60708

In [55]:
#將限制式寫成檔案
opt_mod.write("Electric_test.lp")

# 輸出結果編成Excel

## 1.先將數據分類

In [56]:
others = []  #儲存最佳目標函式值、光伏容量、儲能容量

#儲存充電量、放電量、期末存貨量、購電量、負載量、光伏發電因數、光伏輸出量、購電價
Charge_result, Discharge_result, Inventory_result, Purchase_result, PVoutput_result, Theta_result, Load_result, Price_result = [], [], [], [], [], [], [], []

others.append(["Objective value", round(opt_mod.objVal, 0)])  #最佳目標函式值

for v in opt_mod.getVars():
    if "E_pv" in v.VarName:
        others.append([v.varName, v.x])  #光伏容量
    if "E_bat" in v.VarName:
        others.append([v.varName, v.x])  #儲能容量
    if "Charge" in v.VarName:
        Charge_result.append([v.varName, v.x])  #充電量
    if "Discharge" in v.VarName:
        Discharge_result.append([v.varName, v.x])  #放電量
    if "Purchase" in v.VarName:
        Purchase_result.append([v.varName, v.x])  #購電量
    if "Inventory" in v.VarName:
        Inventory_result.append([v.varName, v.x])  #期末存貨量
    if "PVoutput" in v.VarName:
        PVoutput_result.append([v.varName, v.x])  #光伏輸出量

for month in range(0, 12):
    for date in range(0, N_m[month]):  
        for hour in range(0, 24):
            Theta_result.append(["Theta[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", Theta[month][date][hour]])  #光伏發電因數
            Load_result.append(["Load[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", Load[month][date][hour]])  #負載量
            Price_result.append(["Price[" + str(month+1) + "][" + str(date+1) + "][" + str(hour+1) + "]", Price[month][date][hour]])  #購電價

## 2.把分類數據以日為單位儲存

In [57]:
#將資料以日為單位分類

Others_df, Charge_df, Discharge_df, Inventory_df, Purchase_df, Load_df, Theta_df, PVoutput_df, Price_df =[], [], [], [], [], [], [], [], []

Dataframe = []  #儲存各時段總資料框

index_count = 0  #靠索引值分類，一天24筆資料

for month in range(0, 12):
    for date in range(0, N_m[month]):
        #用pandas的concat方法合併多個資料框 => concat([資料框1, 資料框2, ...], axis)
        #用pandas的DataFrame方法將陣列資料轉換為資料框 => DataFrame(資料陣列, columns = 行名稱)
        Dataframe.append(pd.concat([pd.DataFrame(Charge_result[index_count:index_count+24], columns = ["Charge[month][date][hour]", "Charge"]),
                                    pd.DataFrame(Discharge_result[index_count:index_count+24], columns = ["Discharge[month][date][hour]", "Discharge"]),
                                    pd.DataFrame(Purchase_result[index_count:index_count+24], columns = ["Purchase[month][date][hour]", "Purchase"]),
                                    pd.DataFrame(Inventory_result[index_count:index_count+24], columns = ["Inventory[month][date][hour]", "Inventory"]),
                                    pd.DataFrame(PVoutput_result[index_count:index_count+24], columns = ["PVoutput[month][date][hour]", "PVoutput"]),
                                    pd.DataFrame(Theta_result[index_count:index_count+24], columns = ["Theta[month][date][hour]", "Theta"]),
                                    pd.DataFrame(Load_result[index_count:index_count+24], columns = ["Load[month][date][hour]", "Load"]),
                                    pd.DataFrame(Price_result[index_count:index_count+24], columns = ["Price[month][date][hour]", "Price"])],
                                    axis = 1))  #axis = 1 ==> 橫著方向合併資料
        index_count = index_count +24  #一天存了24筆資料，索引值要加24，代表下一次存下一組24筆資料
        
Others_df = pd.DataFrame(others) #儲存最佳目標函式值、光伏容量、儲能容量

## 3.將366天資料合併為一個資料框

In [58]:
for i in range(1, len(Dataframe)):
    Dataframe[i] = pd.concat([Dataframe[i-1], Dataframe[i]], axis = 0)

## 4.將366天的資料框索引值重設，再與儲存最佳目標函式值、光伏容量、儲能容量的資料框合併

In [59]:
Dataframe[365].reset_index(inplace = True)  #重設索引值 => 0-8783
Dataframe[365] = Dataframe[365].drop(columns = ["index"])  #將舊的索引值刪掉 => 0-24
df = pd.concat([Others_df, Dataframe[365]], axis = 1)  #將366天資料框與儲存最佳目標函式值、光伏容量、儲能容量的資料框合併

## 5.輸出成Excel

In [61]:
#用pandas的to_excel方法匯出成Excel檔，參數是字串型態的檔名
df.to_excel('result/Final_result.xlsx')